## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [2]:
# File location and type
file_location = "/FileStore/tables/clean_diabetes.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

#display(df)
df.show(5)

+-------+----+---+-------+
Glucose| BMI|Age|Outcome|
+-------+----+---+-------+
 148|33.6| 50| 1|
 85|26.6| 31| 0|
 183|23.3| 32| 1|
 89|28.1| 21| 0|
 137|43.1| 33| 1|
+-------+----+---+-------+
only showing top 5 rows

In [3]:
# Create a view or table

temp_table_name = "clean_diabetes_csv"

df.createOrReplaceTempView(temp_table_name)

In [4]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "clean_diabetes_csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [5]:
df = df.withColumn('Glucose',df.Glucose.cast('integer'))
df = df.withColumn('BMI',df.BMI.cast('double'))
df = df.withColumn('Age',df.Age.cast('integer'))
df = df.withColumn('Outcome',df.Outcome.cast('integer'))
df = df.withColumn('label',df.Outcome)
df = df.drop(df.Outcome)
df.show(5)

+-------+----+---+-----+
Glucose| BMI|Age|label|
+-------+----+---+-----+
 148|33.6| 50| 1|
 85|26.6| 31| 0|
 183|23.3| 32| 1|
 89|28.1| 21| 0|
 137|43.1| 33| 1|
+-------+----+---+-----+
only showing top 5 rows

In [6]:
df.describe()

Out[5]: DataFrame[summary: string, Glucose: string, BMI: string, Age: string, label: string]

In [7]:
from pyspark.ml.feature import VectorAssembler
vec_assembler = VectorAssembler(inputCols=['Glucose','BMI','Age'], outputCol='features')

In [8]:
from pyspark.ml import Pipeline
diabetes_pipe  = Pipeline(stages = [vec_assembler])

In [9]:

piped_data = diabetes_pipe.fit(df).transform(df)

In [10]:
train,test = piped_data.randomSplit([.6,.4])

In [11]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression()

In [12]:
import pyspark.ml.evaluation as evals
evaluator = evals.BinaryClassificationEvaluator(metricName='areaUnderROC')

In [13]:
import pyspark.ml.tuning as tune
import numpy as np
grid = tune.ParamGridBuilder()
grid = grid.addGrid(lr.regParam,np.arange(0,.1,.01))
grid = grid.addGrid(lr.elasticNetParam,[0,1])
grid.build();

In [14]:
cv  = tune.CrossValidator(estimator = lr,  estimatorParamMaps=grid,evaluator=evaluator)

In [15]:

best_lr = lr.fit(train)
print(best_lr)

LogisticRegressionModel: uid = LogisticRegression_b32151e21503, numClasses = 2, numFeatures = 3

In [16]:
# Use the model to predict the test set
test_results = best_lr.transform(test)

# Evaluate the predictions
print(evaluator.evaluate(test_results))

0.8148489975901143